## Parámetros

Aquí se resumen los parámetros considerados en el problema, más o menos ordenados. Cuando es posible se adjunta la bibliografía de la que se obtuvo la info.

In [1]:
#Importo librerías
import numpy as np
import math

In [2]:
# parameter	number
# tmax[s]	100e-6
# R0[m]	1e-6
# Rmax[m]	1000e-6
# T0[K]	300
# Tmax[K]	350
# sigmaT[s]	1e-6

### Masa de partículas y moléculas

expresadas en kg

In [3]:
masa_H_molar = 1.00784 #u (unidad de masa atómica). Valor de Wikipedia
masa_H = masa_H_molar*1.66053906660e-27 #kg. La cte de proporcionalidad es la masa de una u, obtenida de NIST Reference https://physics.nist.gov/cuu/Constants/index.html
masa_D_molar = 2.01410178 #u. Valor de Wikipedia
masa_D = masa_D_molar*1.66053906660e-27 #kg.
masa_O_molar = 15.999 #u. Valor de Wikipedia
masa_O = masa_O_molar*1.66053906660e-27 #kg




In [12]:
print(masa_H)
#1.673557692882144e-27

1.673557692882144e-27


### Constantes físico-químicas

NIST Reference: https://physics.nist.gov/cuu/Constants/index.html

In [4]:

Na = 6.02214076e23 #Nro de avogadro [1/mol], obtenida de NIST
K= 1.380649e-23 #constante de Boltzmann [J/K], obtenida de NIST
Rg=Na*K #constante de los gases [J/mol K], obtenida de NIST
 

### Propiedades de gases

In [5]:


#Gas Properties - Argon
a_Ar=0.1345 #constante de Van der Waals del argon []
b_Ar=3.219e-5#constante de Van der Waals del argon []
MAr=0.03942 #masa molar del argon [kg/gmol]
# CvAr=3.0/2.0*Rg #heat capacity at constant volume gas[J/kg K]
k1Ar=0.009 #constantes de la conductividad del gas [W/m K]
k2Ar=3.2e-5
# gama=Rg/(3.0*Rg)+1.0


#Gas Properties - Oxigeno
#a_Ar=0.136; #constante de Van der Waals del argon []
#b_Ar=3.183e-5;#constante de Van der Waals del argon []
MO2=0.032; #masa molar del argon [kg/gmol]
#CvO2=3.0/2.0*Rg; #heat capacity at constant volume gas[J/kg K]
k1O2=0.009; #constantes de la conductividad del gas [W/m K]
k2O2=3.2e-5;

#Gas Properties - Nitrogeno
#a_Ar=0.139; #constante de Van der Waals del argon []
#b_Ar=3.913e-5;#constante de Van der Waals del argon []
MN2=0.028; #masa molar del argon [kg/gmol]
# CvN2=5.0/2.0*Rg; #heat capacity at constant volume gas[J/kg K]
k1N2=669e-5; #constantes de la conductividad del gas [W/m K]
k2N2=6.24357e-5;

### Nro de partículas de cada especie n

Ctes asociadas a las reacciones químimcas

El orden en el vector n_species está dado por  
n[n_species] =      {H2,       H,      O,      O2,     OH,     H2O,    H2O2,   HO2}

In [6]:
n_species = 8 #Nro de especies consideradas
n_reacc = 28 #Nro de reacciones químicas consideradas

#Masas:
masa_species = np.array([2*masa_D, masa_D, masa_O, 2*masa_O, masa_D + masa_O, 2*masa_D + masa_O, 2*masa_D + 2*masa_O, masa_D + 2*masa_O])


### Variables termodinámicas var_termo

In [4]:
n_var_termo = 8
# var_termo = [R, v_R, T, P_B]
# y = [R, v_R, m, mp, T, Tp, pB, pBp]
var_termo = {"R":0, "v_R":1, "m":2, "mp":3, "T":4, "Tp":5, "pB":6, "pBp":7}

In [6]:
y = [1,2,3,4,5,6,7,8]
y[var_termo["v_R"]]

2

### Parámetros para la condensación, evaporación y difusión (REVISAR!!)

In [8]:
#D2O
#Vapor Properties
a_h2o=0.5583 #constante de Van der Waals del agua [m6 Pa/mol]
b_h2o=3.0877e-5 #constante de Van der Waals del agua [m3/mol]
masa_H2O_kgmol= (2*masa_D_molar + masa_O_molar)/1000 #masa molar del gas [kg/mol]. Gabriela había escrito [kg/gmol], qué significa la g?
#    Cvh2o=3.0*Rg #calor especifico a volumen constante del vapor de agua
k1h2o=-0.02028 #constantes de la conductividad del vapor [W/m K]
k2h2o=1.15006e-4
Rv=Rg/masa_H2O_kgmol #constante de los gases para el vapor de agua [J/kg K]
#    beta=1.0
#

#switches del modelo
rapidez = 0 #switch utilizado para tomar a Gama como una funcion de mp, o como constante igual a uno. Si es 1 es rapido, otro numero y es lento
modelo = 2 #Permite elegir el modelo para calcular TB. 1: Yasui, 2: Toegel

#Ctes del modelo de Yasui para calcular TB
n_Yasui=5.0 #constante [1] Yasui
ap=0.827 #constante [1] Yasui
sigmap=0.4e-18 #[] Yasui

#Liquid-vapor properties
# L=2258987.94;# H2O calor latente de vaporizacion [J/kg]
# L=2066700.0;# D2O calor latente de vaporizacion [J/kg]
alfae=1.0; #coeficiente de acomodacion termico [1]
alfaM=0.1; #[1]#coeficiente de acomodacion para los procesos de evaporación o condensación

#Parámetros del agua a distintas temperaturas
#Los siguientes parámetros dependen de la temperatura. Los copié del código de Gabriela a:
Tinf=35+273.15 #D2O
rol=1101.8 #densidad del liquido [kg/m3]. Necesario para calcular pB
mu=0.000869  #viscosidad [Pa s]. Necesario para calcular pB
# cl=1415.1 #velocidad del sonido del liquido [m/s]
# cpl=4236.50#calor especifico del liquido [J/kg K]
# kl=0.60562#conductividad termica del agua [W/m K]
sigma=0.07031  #tension superficial [J/m2]. Necesario para calcular pB
# alfa=kl/cpl/rol
pvap0=4949 #este parámetro depende de T_nf y cambia si es D2O o H2O. Yo copié el de D2O. Es importante en la fórmual 2.2.1.4 para calcular m_punto



#Masas Molares D2O
#Vector de las masas de las especies expresadas en Kg/mol
masa_species_Kgmol = masa_species/(1.66053906660e-27)/1000

#ANTES: CÓDIGO DE GABRIELA
#Estos parámetros se emplean en la función MasaMolar().
# MH2=0.004
# MH=0.002
# MO=0.016
# MOH=0.018
# MN=0.014
# MNH=0.015
# MNH2=0.016
# MNH3=0.017
# MN2H=0.029
# MNO=0.03
# MNO2=0.046
# MN2O=0.044
# MHNO=0.031
# MH2O2 = 0.034
# MHO2 = 0.033





#### Conductividades térmicas

In [9]:
#Conductividades
#ordenadas al origen
k1H2=k1O2 
k1H=k1O2
k1O=k1O2
k1OH=k1O2
k1N=k1O2
k1NH=k1O2
k1NH2=k1O2
k1NH3=k1O2
k1N2H=k1O2
k1NO=k1O2
k1NO2=k1O2
k1N2O=k1O2
k1HNO=k1O2
k1O2=k1O2

#pendientes (respecto de la temperatura: k1i+k2i*T)
k2H2=k2O2 
k2H=k2O2
k2O=k2O2
k2OH=k2O2
k2N=k2O2
k2NH=k2O2
k2NH2=k2O2
k2NH3=k2O2
k2N2H=k2O2
k2NO=k2O2
k2NO2=k2O2
k2N2O=k2O2
k2HNO=k2O2

### Parámetros de la evolución

Por lo pronto se considera que el radio de la burbuja y la temperatura evolucionan a través de funciones conocidas


In [10]:
#Parámetros de la evolución química del sistema
tmax = 100e-6 #1.833973816e-010 // 1e5 //100e-6
R0 = 1e-6
Rmax = 1000e-6
T0 = 300
Tmax = 6000
sigmaT = 1e-6



def R(t):
    return ((Rmax-R0)/tmax*t  + R0)

def v_R(t):
    return (Rmax-R0)/tmax

def Vol(R):
    return 4/3 * math.pi * R**3

def dVoldt(t):
    return 0
    #return 4*Pi*pow(R(t),2)*(Rmax-R0)/tmax;

#Mantengo esta def para la parte de reacciones:
# def V(t):
#     return 4/3 * math.pi * R(t)**3

# def dVdt(t):
#     return 0
#     #return 4*Pi*pow(R(t),2)*(Rmax-R0)/tmax;

def T(t):
    return (Tmax-T0)*np.exp(-0.5*pow(t/sigmaT,2)) + T0



### Parámetros del método numérico

In [11]:
#Temporal integration control
# eps=1.0e-7;#error
# step=1.0e-11; #Time step [s]
# hmin=1.0e-16; #el paso minimo que debe tener en cuenta
# numciclos=1; #numero de ciclos que calcula NO USADO
epsilon=1.0e-6; #Convergence criterion para las cantidades que se calculan con el metodo de biseccion[1]. Lo necesito para, entre otras cosas, calcular m_punto.
# N=100; # cantidad de pasos utilizados para integrar la funcion error (ERF()). No lo uso porque en Python uso directamente la función math.erf() para calcular la función error.

#Parámetros para bisección
#Limits
Tmin=200.0 #Minimum temperature para el calculo de biseccion [K]
Tmax=20000.0 #Maximum temperature para el calculo de biseccion [K]

#Para calcular mp en la fórmula 2.2.1.3 de la tesis de Gabriela
mpmin=-5.5e4 #Minimum mp para el calculo de biseccion [kg/m2 s]
mpmax=5.5e4 #Maximum mp para el calculo de biseccion [kg/m2 s]
epsilon=1.0e-6; #Convergence criterion para las cantidades que se calculan con el metodo de biseccion[1]. Lo necesito para, entre otras cosas, calcular m_punto.


# Cmin=0.0
# Cmax=4.0
# Rmax=1.0e-6